In [1]:
import pandas as pd
import numpy as np
import random
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
import matplotlib.pyplot as plt
import os

## Preprocessing images

#### Loading previous dataset

In [2]:
# Creating an array of positive and negative labelled datasets

sample_data_limit = 200

all_data = pd.read_csv("sample_data.csv")
numpy_array = all_data.values

positives, negatives = ([], [])

iter = 0
while(len(positives) < sample_data_limit or len(negatives) < sample_data_limit):
    label = numpy_array[iter][3]
    if label == 1:
        positives.append(tuple(numpy_array[iter][1:]))
    else:
        negatives.append(tuple(numpy_array[iter][1:]))
        
    iter += 1



#### Resize images to (105, 105) and scale from 0-255 to 0-1

In [3]:
# function to convert image to (105, 105)
def preprocess(image_path):
    byte_image = tf.io.read_file(image_path)
#     print(byte_image)
    img = tf.io.decode_jpeg(byte_image)
#     print(img)
#     plt.imshow(img)
    img = tf.image.resize(img, (105, 105))

    img = img / 255.0 
#     plt.imshow(img)
    #     plt.n
    return img

### Train and Test Split

In [6]:
split_ratio = 0.7

# Training partition
train_data = data.take(round(len(data)*split_ratio))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# Testing partition
test_data = data.skip(round(len(data)*split_ratio))
test_data = test_data.take(round(len(data)*(1-split_ratio)))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)


## Building Model

#### Making Embedding Layer

In [7]:
def make_embedding(): 
    inp = Input(shape=(105,105,3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

embedding = make_embedding()
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 21, 21, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 128)       26

#### Making Distance Layer

In [8]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [9]:
l1 = L1Dist()

In [10]:
# train_data_getter = train_data.as_numpy_iterator()
# train_sample = train_data_getter.next();

#### Making Siamese Model

In [15]:
# siamese_model = make_siamese_model()

siamese_model = tf.keras.models.load_model('siamese_modelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [16]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 105, 105, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 105, 105, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

#### Training

##### Setting up loss and optimizer

In [17]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [18]:
opt = tf.keras.optimizers.legacy.Adam(1e-4) # 0.0001

##### Checkpoints

In [19]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

##### Train step function

##### Training Loop

In [29]:
from tensorflow.keras.metrics import Precision, Recall

##### Train model

In [31]:
EPOCHS = 50

#### Evaluating Model


In [34]:
from tensorflow.keras.metrics import Precision, Recall

In [44]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [38]:
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

[1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1]

In [39]:
y_true

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1], dtype=int32)

#### Calculating Metrics

In [41]:
m= Recall()

In [42]:
m.update_state(y_true, y_hat)

In [43]:
m.result().numpy()

1.0

In [44]:
m = Precision()

In [45]:
m.update_state(y_true, y_hat)

In [46]:
m.result().numpy()

1.0